In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
covid_data = pd.read_csv('covid_data_clean.csv')
countries_data = pd.read_csv('countries_data_clean.csv',decimal=',')

In [4]:
covid_data.head(10)

,Unnamed: 0,Country/Region,Date,Confirmed,Deaths,Recovered,Ellapsed Days
0,0,Afghanistan,2020-02-24,1,0,0,0
1,1,Afghanistan,2020-02-25,1,0,0,1
2,2,Afghanistan,2020-02-26,1,0,0,2
3,3,Afghanistan,2020-02-27,1,0,0,3
4,4,Afghanistan,2020-02-28,1,0,0,4
5,5,Afghanistan,2020-02-29,1,0,0,5
6,6,Afghanistan,2020-03-01,1,0,0,6
7,7,Afghanistan,2020-03-02,1,0,0,7
8,8,Afghanistan,2020-03-03,1,0,0,8
9,9,Afghanistan,2020-03-04,1,0,0,9


In [5]:
countries_data.head(10)

,Unnamed: 0,Country,Population,Pop. Density (per sq. mi.),Infant mortality (per 1000 births),GDP ($ per capita),Deathrate
0,0,Afghanistan,31056997,48.0,163.07,700.0,20.34
1,1,Albania,3581655,124.6,21.52,4500.0,5.22
2,2,Algeria,32930091,13.8,31.00,6000.0,4.61
3,3,Samoa,57794,290.4,9.27,8000.0,3.27
4,4,Andorra,71201,152.1,4.05,19000.0,6.25
5,5,Angola,12127071,9.7,191.19,1900.0,24.20
6,7,Antigua and Barbuda,69108,156.0,19.46,11000.0,5.37
7,8,Argentina,39921833,14.4,15.18,11200.0,7.55
8,9,Armenia,2976372,99.9,23.28,3500.0,8.23
9,11,Australia,20264082,2.6,4.69,29000.0,7.51


In [6]:
#Merge the two columns together and drop uncessary columns
merged_inner = pd.merge(left=covid_data, right=countries_data, left_on='Country/Region', right_on='Country').drop(columns=['Country', 'Date', 'Unnamed: 0_x', 'Unnamed: 0_y'])

#Drop rows with at least one missing value
merged_inner.dropna(inplace=True)

def createPrevValues(dataset, originalColumnName, commonColumnName, newColumnName):
    dataset[newColumnName] = dataset.loc[merged_inner[commonColumnName].shift(-1)==dataset[commonColumnName], originalColumnName]
    dataset[newColumnName] = dataset[newColumnName].shift()
    dataset[newColumnName].fillna(0, inplace=True)

#Get cases of previous day
createPrevValues(merged_inner, 'Confirmed', 'Country/Region', 'Prev_Confirmed')

#Get deaths of previous day
createPrevValues(merged_inner, 'Deaths', 'Country/Region', 'Prev_Deaths')

#Get Recovered of previous day
createPrevValues(merged_inner, 'Recovered', 'Country/Region', 'Prev_Recovered')

merged_inner.head()

,Country/Region,Confirmed,Deaths,Recovered,Ellapsed Days,Population,Pop. Density (per sq. mi.),Infant mortality (per 1000 births),GDP ($ per capita),Deathrate,Prev_Confirmed,Prev_Deaths,Prev_Recovered
0,Afghanistan,1,0,0,0,31056997,48.0,163.07,700.0,20.34,0.0,0.0,0.0
1,Afghanistan,1,0,0,1,31056997,48.0,163.07,700.0,20.34,1.0,0.0,0.0
2,Afghanistan,1,0,0,2,31056997,48.0,163.07,700.0,20.34,1.0,0.0,0.0
3,Afghanistan,1,0,0,3,31056997,48.0,163.07,700.0,20.34,1.0,0.0,0.0
4,Afghanistan,1,0,0,4,31056997,48.0,163.07,700.0,20.34,1.0,0.0,0.0


In [7]:
#Get set with the values that the model receives
X = merged_inner.drop(columns=['Country/Region', 'Confirmed'])

#Values the model should predict
y = merged_inner['Confirmed']

In [8]:
#Get the split of the trainning and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
#Fitting model with tranning set
from sklearn import svm
regressor = svm.SVR(kernel='sigmoid')
regressor.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='sigmoid', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [10]:
#Predicting
y_pred = regressor.predict(X_test)

In [11]:
#Show difference of real and predicted values
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

,Actual,Predicted
6116,101,107.666459
9997,4,95.768807
2917,189,138.393098
9731,10801,219.563333
1386,5,114.213935
...,...,...
9057,2,145.093022
6871,717,270.960445
10935,1,189.965223
7863,167,107.939221


In [12]:
#Show Errors
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 8606.517321365689
Mean Squared Error: 2755610217.914102
Root Mean Squared Error: 52493.906483649145


In [13]:
#Values to compare with error -> less than 10% of the mean result is a good success
merged_inner.describe()

,Confirmed,Deaths,Recovered,Ellapsed Days,Population,Pop. Density (per sq. mi.),Infant mortality (per 1000 births),Deathrate,Prev_Confirmed,Prev_Deaths,Prev_Recovered
count,1.158100e+04,11581.000000,11581.000000,11581.000000,1.158100e+04,11581.000000,11581.000000,11581.000000,1.158100e+04,11581.000000,11581.000000
mean,8.500861e+03,561.370866,2332.912184,34.777653,4.841404e+07,293.000466,35.008250,9.472936,8.169845e+03,538.116484,2224.839133
std,5.111700e+04,3537.549173,12026.504142,23.305229,1.635563e+08,1390.625236,35.220779,4.699340,4.955015e+04,3425.930500,11632.311556
min,1.000000e+00,0.000000,0.000000,0.000000,2.925100e+04,1.800000,2.290000,2.410000,0.000000e+00,0.000000,0.000000
25%,1.500000e+01,0.000000,0.000000,16.000000,3.102229e+06,29.300000,7.030000,6.210000,1.300000e+01,0.000000,0.000000
50%,1.380000e+02,2.000000,12.000000,32.000000,9.690222e+06,77.400000,20.970000,8.300000,1.240000e+02,2.000000,11.000000
75%,1.445000e+03,31.000000,201.000000,50.000000,3.293009e+07,182.200000,55.510000,11.110000,1.380000e+03,28.000000,177.000000
max,1.257023e+06,75662.000000,195036.000000,106.000000,1.313974e+09,16271.500000,191.190000,29.740000,1.228603e+06,73431.000000,189910.000000
